In [1]:
import numpy as np

import astropy.units as u
from astroquery.gaia import Gaia
from astropy.table import Table, Column
from astropy.io import ascii
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

In [2]:
table = Gaia.load_table('gaiaedr3.gaia_source')

Retrieving table 'gaiaedr3.gaia_source'


In [3]:
SPEC_CAT = ascii.read("../reference_data/color_table.txt").to_pandas()

def gaia_sptype(color, cat=SPEC_CAT, color_name='G-Rp'):
    diff = cat[color_name] - color
    absdiff = abs(diff)
    sptype = cat.loc[absdiff.idxmin()]['SpT'].replace('V', '')
    return sptype

In [4]:
for col in table.columns:
    print(col.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

In [56]:
def gaia2mmt(low_mag, high_mag, every=10, outfile=None):
    """
    Grab stars from Gaia EDR3 in the range between low_mag and high_mag. The 'every' parameter can be used to 
    limit the size of the sample, e.g. to select every 10th star. This uses the random_index in the Gaia table to
    assure sampling is representative. For MMT catalogs, about 15000-20000 entries is about the limit of what the
    scope gui can comfortably handle. The output table is configured to output an MMT-compatible catalog directly. 
    If outfile is not None, the catalog will be written to outfile. 
    """

    job = Gaia.launch_job_async(
        "select all designation,ref_epoch,ra,dec,pmra,pmdec,phot_g_mean_mag as gmag,g_rp,random_index "
        f"from gaiaedr3.gaia_source where (phot_g_mean_mag > {low_mag} AND phot_g_mean_mag < {high_mag} AND dec > -40 AND "
        f"pmra IS NOT NULL AND MOD(random_index, {every}) = 0) order by gmag"
    )
    r = job.get_results()
    r['sptype'] = [gaia_sptype(g_rp) for g_rp in r['g_rp']]
    ids = [f"GEDR3_{s.split()[-1]}" for s in r['designation']]
    t_obs = Time(r['ref_epoch'].data, format='decimalyear')
    t_ref = Time('J2000')
    c = SkyCoord(
        ra=r['ra'],
        dec=r['dec'],
        pm_ra_cosdec=r['pmra'],
        pm_dec=r['pmdec'],
        obstime=t_obs,
        frame='icrs'
    )
    c_j2000 = c.apply_space_motion(t_ref)
    ra_2000 = c_j2000.ra.to_string(unit=u.hr, sep=":", precision=4, pad=True)
    dec_2000 = c_j2000.dec.to_string(unit=u.deg, sep=":", alwayssign=True, precision=3, pad=True)
    pmra_mmt = r['pmra'].data / 15. / 10. / np.cos(c_j2000.dec)  # MMT uses seconds of time per century
    pmdec_mmt = r['pmdec'].data / 10.  # MMT uses seconds of arc per century

    out_tab = Table()
    out_tab['ID'] = Column(ids, format='<25')
    out_tab['RA'] = Column(ra_2000)
    out_tab['Dec'] = Column(dec_2000)
    out_tab['PM_RA'] = Column(pmra_mmt.data, format=".4f")
    out_tab['PM_Dec'] = Column(pmdec_mmt.data, format=".4f")
    out_tab['Mag'] = Column(r['gmag'].data, format=".2f")
    out_tab['Sp_Type'] = Column(r['sptype'])
    out_tab.add_column('J2000.0', name='Epoch')
    if outfile is not None:
        out_tab.write(outfile, format='ascii.fixed_width_no_header', delimiter=None, overwrite=True, bookend=False)
    return out_tab

In [74]:
t = gaia2mmt(low_mag=14, high_mag=15, every=750, outfile="../catalogs/gaia_14_15.cat")

INFO: Query finished. [astroquery.utils.tap.core]


/home/tim/conda/envs/py39/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 18936 of "distance overridden (Note 6)", 3 of "binary logical OR of the above warnings"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [75]:
t

ID,RA,Dec,PM_RA,PM_Dec,Mag,Sp_Type,Epoch
str25,str13,str13,float64,float64,float32,str4,str7
GEDR3_2816265379461572736,22:55:37.1313,+14:36:22.038,-0.0325,-0.6195,14.00,K1.5,J2000.0
GEDR3_4052360128432806528,18:19:41.0484,-27:15:33.609,-0.0359,-1.2902,14.00,K6.5,J2000.0
GEDR3_3955227759319992448,12:50:30.2464,+22:40:21.809,0.0060,-0.5510,14.00,G8,J2000.0
GEDR3_2062180379135558656,20:12:58.6533,+39:29:51.607,0.0265,-0.6461,14.00,K0,J2000.0
GEDR3_1753823236463891968,20:26:42.3125,+10:49:29.796,-0.0229,-0.4863,14.00,K4,J2000.0
GEDR3_2033125028482266752,19:38:02.5425,+31:11:09.791,-0.0198,-0.6800,14.00,K6,J2000.0
GEDR3_5708422723670675584,08:37:15.0455,-18:42:50.655,-0.0209,0.4743,14.00,F9,J2000.0
GEDR3_4167741950856812672,17:28:37.6488,-08:22:54.554,-0.0106,-0.5045,14.00,M2,J2000.0
GEDR3_3341861005881158400,06:04:44.6401,+10:49:16.631,-0.0045,-0.8632,14.00,K1.5,J2000.0


In [52]:
r

designation,ref_epoch,ra,dec,pmra,pmdec,gmag,g_rp,random_index
,yr,deg,deg,mas / yr,mas / yr,mag,mag,
object,float64,float64,float64,float64,float64,float32,float32,int64
Gaia EDR3 5616982973026239744,2016.0,111.24600126675567,-24.426311998237164,-8.501098184244631,5.64171298412784,8.000086,-0.12911797,1167574125
Gaia EDR3 2052508903053556224,2016.0,293.9017224692132,40.02732792622806,-13.4830390010105,-23.118987591454776,8.000522,0.48545504,631342970
Gaia EDR3 6543567500784109184,2016.0,345.49293591514936,-39.75612587699399,20.35466200288655,-34.190271789146976,8.000678,0.37519026,935189690
Gaia EDR3 5144850137428522240,2016.0,34.17715005312012,-16.281429380194407,22.598018213010846,-3.422430289632752,8.001075,0.910161,1810319205
Gaia EDR3 1707200365734544256,2016.0,233.1264821274129,76.77390165748176,30.280228251154064,56.74175883453533,8.001079,0.527565,984254395
Gaia EDR3 3162328486688220416,2016.0,114.21619604117961,12.078753444303207,-11.901180972336695,-4.530833432719433,8.001189,0.2581854,197204900
Gaia EDR3 6907170258014433664,2016.0,310.36227897409674,-7.196142480455991,4.8912190325795395,-19.42137966472189,8.001359,0.65695477,439956450
Gaia EDR3 1828606932155119360,2016.0,304.3331750950117,20.226907916649587,-4.715339460188449,-6.54964065285932,8.001439,0.7537589,127151070
